### Name :  W16 作業2
### Function : Application of NER and Topic Model 

#### input data : 許淑華.csv

In [19]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import gc
import math
import jieba
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter

##  function Extract_NER():   Extract NER from receiving text list

In [20]:
def Extract_NER(ls_textRow):
    from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
    
    # 載入 自訂詞典
    User_Dict = {}
    with open("KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
        for udic in UDicts:
            udWord = udic.strip().split(" ")
            if len(udWord) > 1:
                User_Dict[udWord[0]] = udWord[1]
            else:
                User_Dict[udWord[0]] = 10    # 未給定權重者一律賦予預設值 10                
    dictionary = construct_dictionary(User_Dict)
 
    ws = WS("./data")
    pos = POS("./data")
    ner = NER("./data")
    
    #  ls_segRow = ws(ls_textRow) ------------   without User Dictionary
    ls_segRow = ws(ls_textRow, 
                   sentence_segmentation=True,
                   segment_delimiter_set = {",", "。", ":", "?", "!", ";", "、"},
                   coerce_dictionary = dictionary)
    #print("ls_segRow = ", ls_segRow)
    #print("-----------------------------")
    
    del ws
    del WS
    gc.collect()
    ls_posRow = pos(ls_segRow)
    del pos
    del POS
    gc.collect()
    ls_nerRow = ner(ls_segRow,ls_posRow)
    del ner
    del NER
    gc.collect()
    #print("ls_nerRow = ", ls_nerRow)
    #print("-----------------------------")
    return ls_nerRow

## 1. Read data from csv or xlsx file

In [21]:
df = pd.read_csv('許淑華.csv',encoding='utf-8') #匯資料
df.head(1)

,Unnamed: 0,屆期,會期,會議日期,委員姓名,逐字稿
0,4191,8,7,3/18/2015,許淑華,﻿許委員淑華：主席、各位列席官員、各位同仁。今天上午有許多委員都很關心路平專案，路平專案\n...


## 2.  Main function:
### 2.1  Pass text data to function "Extract_NER" by row
### 2.2  Integrate every 10 NER return rows into one data frame  
### 2.3  Write frame into small csv after droping duplicate, locating class, filtering short NER, sorting by class.. 

In [11]:
#逐筆傳遞欄位 "text"
hsu_text = df["逐字稿"]
print("逐字稿 欄位筆數 = ",len(hsu_text))
print("------------------------------------------")
ls_class = list()
ls_ner = list()
fileCunt = 0
rowCunt = 0
for i in range(len(hsu_text)):
    rowCunt += 1
    ls_dfRow = [hsu_text[i]]
    # print("ls_dfRow = ", ls_dfRow)
    print("Row No = ",str(i))
    entity_list = Extract_NER(ls_dfRow)
    for j in range(len(entity_list[0])):
        seg = entity_list[0].pop()
        # print("seg = ",seg)
        ls_class.append(seg[2])
        ls_ner.append(seg[3])
    #print("===========================================")
    #print("ls_class = ",ls_class)
    #print("ls_ner = ",ls_ner)
    # if i == 5 or rowCunt == 2:
    if i == (len(hsu_text)-1) or rowCunt == 10:
        fileCunt += 1
        extract_df = pd.DataFrame({'Class':ls_class, 'NER':ls_ner})
        deDup_df = extract_df.drop_duplicates().copy()
        del extract_df #去除重複
        cate_df = deDup_df.loc[deDup_df['Class'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE','NORP'])].copy()
        del deDup_df
        filt_df = cate_df[cate_df['NER'].map(len) >= 2].copy()
        del cate_df
        NER_df = filt_df.sort_values(by=['Class']).copy()
        del filt_df
        #print("__________________________________________________________")  
        #print(NER_df)
        print("fileCunt No = ",str(fileCunt))
        NER_df.to_csv(r"Ner out/NewsNER870_" + str(fileCunt)+ ".csv")
        #NER_df.to_csv(r"ner out/NerTest_" + str(fileCunt)+ ".csv")
        rowCunt = 0
print("=====================================================================")
print("Totle NER Files = ",fileCunt)


逐字稿 欄位筆數 =  288
------------------------------------------
Row No =  0


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  1
Row No =  2
Row No =  3
Row No =  4
Row No =  5
Row No =  6
Row No =  7
Row No =  8
Row No =  9
fileCunt No =  1
Row No =  10


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  11
Row No =  12
Row No =  13
Row No =  14
Row No =  15
Row No =  16
Row No =  17
Row No =  18
Row No =  19
fileCunt No =  2
Row No =  20


...

Row No =  261
Row No =  262
Row No =  263
Row No =  264
Row No =  265
Row No =  266
Row No =  267
Row No =  268
Row No =  269
fileCunt No =  27
Row No =  270


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  271
Row No =  272
Row No =  273
Row No =  274
Row No =  275
Row No =  276
Row No =  277
Row No =  278
Row No =  279
fileCunt No =  28
Row No =  280


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  281
Row No =  282
Row No =  283
Row No =  284
Row No =  285
Row No =  286
Row No =  287
fileCunt No =  29
Totle NER Files =  29


## 3. Integrate all csv files into single data frame

In [13]:
import glob
# import pandas as pd

path = r'Ner out'
#path = r'ner out'
allFiles = glob.glob("{}/*.csv".format(path))
li = []
for filename in allFiles:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

nerFrame = pd.concat(li, axis=0, ignore_index=True)
print(len(nerFrame))
nerFrame.groupby('Class').count()

18927


,NER
Class,
EVENT,478
GPE,3909
LAW,765
LOC,1036
NORP,788
ORG,7129
PERSON,4822


## 4. Drop duplicates, group by class and write into csv file 

In [14]:
ddFrame = nerFrame.drop_duplicates().copy()
print(len(ddFrame))
ddFrame.groupby('Class').count()

1097


,NER
Class,
EVENT,33
GPE,202
LAW,46
LOC,76
NORP,38
ORG,437
PERSON,265


In [16]:
ddFrame.sort_values(by=['Class']).to_csv(r"Ner out/Ligis_NER.csv")
#ddFrame.sort_values(by=['Class']).to_csv(r"ner out/NerTest.csv")

#### 用許淑華的資料跑TOPIC MODEL

In [22]:
text_list = list(df['逐字稿']) # 把所有message轉成一個list 用for iterate

hsu_str = ','.join(text_list)
print(hsu_str) #轉字串斷詞

﻿許委員淑華：主席、各位列席官員、各位同仁。今天上午有許多委員都很關心路平專案，路平專案
係由地方政府制定，由地方政府來執行。主委剛剛提到，我們在 92 年的時候，訂定了共同管線

開挖標準，處長也說了我們訂有罰責；但是我在這裡也要為地方政府講話，地方政府是開挖單

位，他們不會莫名其妙的去開挖，就算訂了罰責，罰款也很少，主要是他們必須有開挖的需求
，才會提出申請。

現在針對共同管線的開挖，我們制訂了一些新的政策，包括新開闢的道路，就可以來實施，

但是對舊有道路，我們可能還沒有辦法做到，可否請主委和內政部及地方政府商量，將相對性

的預算提高？因為道路路平都是由地方政府及內政部營建署來編列預算，公共工程委員會本身

是監督單位，必須在工程品質上做相對性的要求，而路平專案又是民眾最有感受的事情，所以

請公共工程委員會與相關單位協商，提高維護費用這個部分。尤其近年來，因為道路維護不易
，引發國賠問題，相關的金額大概在 1,500 萬至 3,000 萬之間，都必須由政府去負擔。針對這個
部分，也要請主委與相關單位做一些基本的努力。

其次，有關政府採購法的部分，現在小額採購在 10 萬以下，是不需要經過招標的，但是在地

方政府整個運作下，因為很多廠商就近服務比較方便，所以，我們常看到地方政府在搶救過程
中，會把很多工程拆成 10 萬以下的工程，這對道路的品質和整個作業來說，其實是不夠完整的

。主委有沒有考慮過，提高地方首長小額採購的金額，不要讓每一個工程都分成細小的工程來
進行？

主席：請工程會許主任委員答復。

許主任委員俊逸：主席、各位委員。對於小額採購的金額是不是要提高，我們當然可以來研究；但

不管有沒有提高，它還是可以執行，我們可以訂一個開口合約。以前我在管道路的時候，也是
訂開口合約，在緊急搶修的時候，就可以利用開口合約，依單價實做實算，給付廠商。

許委員淑華：沒錯，雖然有訂定開口契約以爭取搶救時間，但地方首長能否就這個空間作更好的運
用？不只是很多小型的工程，其實 10 萬塊也做不了很多工作，雖然應該防弊，但也不需要將地
方政府人員，包括行政首長當成小偷一樣來認定，我覺得小額採購部分規定為 10 萬元確嫌嚴苛

，如果讓地方政府有更寬鬆的金額限制以為裁決，我相信對地方政府推動公共事務上是有幫助
的。

許主任委員俊逸：我們有給地方政府權限，對 

In [23]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import gc
import tensorflow as tf
# import keras

# 載入 ckiptagger 自訂詞典
User_Dict = {}
with open("KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
    for udic in UDicts:
        udWord = udic.strip().split(" ")
        if len(udWord) > 1:
            User_Dict[udWord[0]] = udWord[1]
        else:
            User_Dict[udWord[0]] = 100    # 未給定權重者一律賦予預設值 100               
dictionary = construct_dictionary(User_Dict)

In [24]:
# 載入模型
ws = WS("./data")
# pos = POS("./data")
# ner = NER("./data")

C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [25]:
# 載入 StopWord
stopword_list = []
with open("KCC Data/Dict/KccStopWord2020.txt","r", encoding='utf-8-sig') as stopwords:
    for stopword in stopwords:
        stopword_list.append(stopword.strip())

In [26]:
# 開始斷詞
# sentence_segmentation = True,  => To consider delimiters
# segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
# recommend_dictionary = dictionary1, # words in this dictionary are encouraged
# coerce_dictionary = dictionary2, # words in this dictionary are forced
strCut = ws([hsu_str], sentence_segmentation=True,
                 segment_delimiter_set = {'"',"\r\n","，","「","」",",","？",'\n', "。", ":", "?", "!", ";", "、"},
                 coerce_dictionary = dictionary)

print("------------CKIPTagger Word Segment ws(hsu_str,....) + 刪除標點及停用字 ---------------")
stayed_lst= [] #斷好字的
for word in strCut[0]:
     # 刪除停用字
    if word not in stopword_list:
        stayed_lst.append(word)
        
print(" ".join(stayed_lst))

------------CKIPTagger Word Segment ws(hsu_str,....) + 刪除標點及停用字 ---------------
﻿ 許 委員 淑華 列席 官員 位 同仁 上午 委員 關心 路平 專案 路平 專案 
 係 地方 政府 制定 地方 政府 執行 主委 剛剛 提到  92  年 訂定 管線 
 
 開挖 標準 處長 說 訂有 罰責 地方 政府 講話 地方 政府 開挖 單
 
 位 莫名其妙 開挖 訂 罰責 罰款 開挖 需求 
 提出 申請 
 
 管線 開挖 制訂 新 政策 包括 新 開闢 道路 實施 
 
 舊有 道路 辦法 可否 請 主委 內政部 地方 政府 商量 相對性 
 
 預算 提高 道路 路平 地方 政府 內政部營建署 編列 預算 公共工程委員會 
 
 監督 單位 工程品質 做 相對性 路平 專案 民眾 感受 
 
 請 公共工程委員會 相關 單位 協商 提高 維護 費用 近年 道路維護 不易 
 引發 國賠 相關 金額  1 500  萬  3 000  萬 之間 政府 負擔 
 請 主委 相關 單位 做 努力 
 
 政府採購法 小額 採購  10 萬 需要 招標 地
 
 方政府 整 運作 廠商 就近 服務 地方 政府 搶救 過程 
 中 工程 拆成  10 萬 工程 道路 品質 作業 說 夠 完整 
 
 主委 有沒有 提高 地方 首長 小額 採購 金額 工程 分成 細小 工程 
 
 
 請 工程 許 主任委員 答復 
 
 許 主任委員 俊逸 委員 小額 採購 金額 提高 研究 
 
 有沒有 提高 執行 訂 開口 合約 道路 
 訂 開口 合約 緊急 搶修 利用 開口 合約 單價 實 做 實算 給付 廠商 
 
 許 委員 淑華 錯 訂定 開口 契約 搶救 時間 地方 首長 空間 作 運
 小型 工程  10  萬 塊 做不了 工作 防 弊 需要 地
 方政府 人員 包括 行政 首長 當成 小偷 認定 小額 採購 規定為  10 萬 元 確嫌 嚴苛 
 
 地方 政府 寬鬆 金額 裁決 地方 政府 推動 公共事務 幫助
 
 
 許 主任委員 俊逸 有給 地方 政府 權限  100  萬 地方 首長 權限 訂定 
 
 具 明 理由 小額 額度 提高 涉及 修法 修法 相關 機關 協商 
 作 修

In [31]:
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel

In [32]:
fList = []
with open("許淑華_output.txt","r", encoding='utf-8-sig') as files:
     for file in files:
        #simple_preprocess: tokenize,去除標點 - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15) 
        fList.append(list(utils.simple_preprocess(file, deacc=True, min_len=2,)))
print("fList[:2] = ", fList[:2])
print("------------------------------")

fList[:2] =  [['jieba_results'], ['委員', '淑華', '主席', '各位', '列席', '官員', '各位', '同仁', '今天上午', '許多', '委員', '關心', '專案', '專案', '地方', '政府', '制定', '地方', '政府', '執行', '主委', '剛剛', '提到', '我們', '時候', '訂定', '共同', '管線', '開挖', '標準', '處長', '我們', '訂有', '罰責', '但是', '這裡', '地方', '政府', '講話', '地方', '政府', '開挖', '他們', '不會', '莫名其妙', '開挖', '就算', '罰責', '罰款', '很少', '主要', '他們', '必須', '開挖', '需求', '提出申請', '現在', '針對', '共同', '管線', '開挖', '我們', '制訂', '一些', '政策', '包括', '開闢', '道路', '可以', '實施', '但是', '舊有', '道路', '我們', '可能', '沒有', '辦法', '做到', '可否', '主委', '內政部', '地方', '政府', '商量', '相對性', '預算', '提高', '因為', '道路', '路平', '地方', '政府', '內政部', '營建', '署來', '編列', '預算', '公共', '工程', '委員會', '本身', '監督', '單位', '必須', '工程', '品質', '相對性', '要求', '而路平', '專案', '民眾', '最有', '感受', '事情', '所以', '公共', '工程', '委員會', '相關', '單位', '協商', '提高', '維護費用', '這個', '部分', '尤其', '近年來', '因為', '道路', '維護', '不易', '引發', '國賠', '問題', '相關', '金額', '大概', '萬至', '之間', '必須', '政府', '負擔', '針對', '這個', '部分', '主委', '相關', '單位', '一些', '基本', '努力', '其次', '有關', '政府', '採購', '部分', '現在', '小額', '採

In [33]:
dict = corpora.Dictionary(fList)
print("dict = ", dict)
print("------------------------------")

dict =  Dictionary(18387 unique tokens: ['jieba_results', '一些', '一個', '一定', '一樣']...)
------------------------------


In [34]:
# creat mapping matrix of [word_id, word_frequency] for each word segment
corpus = [dict.doc2bow(text) for text in fList]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:1]]
print("======================================================")

corpus[:1] =  [[(0, 1)]]
------------------------------
print lists of [word:frequency]
('jieba_results', 1)


In [35]:
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=10, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("KCC Data/NewsCisTest-KccDict2020-LDA-TopicNum10_hsu.txt", 'w', encoding='utf-8-sig')
lstTM = lda_model.print_topics(num_topics=10, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(fList), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)

strTM =  00.000*"拿藥" + 0.000*"會型" + 0.000*"但還" + 0.000*"慢性" + 0.000*"快滿" + 0.000*"常多" + 0.000*"家健保" + 0.000*"奶粉" + 0.000*"各司其職" + 0.000*"吃藥" + 0.000*"另一家" + 0.000*"取藥" + 0.000*"去領" + 0.000*"加一層" + 0.000*"分業" + 0.000*"出還" + 0.000*"公會" + 0.000*"有例" + 0.000*"箋會" + 0.000*"第十二條"

strTM =  10.023*"我們" + 0.018*"部長" + 0.013*"交通部" + 0.012*"現在" + 0.011*"觀光" + 0.010*"佳龍" + 0.010*"委員" + 0.009*"所以" + 0.009*"因為" + 0.009*"可以" + 0.009*"一個" + 0.008*"局長" + 0.008*"淑華" + 0.007*"次長" + 0.006*"這個" + 0.006*"補助" + 0.006*"很多" + 0.005*"就是" + 0.005*"公路" + 0.005*"目前"

strTM =  20.003*"無記名" + 0.003*"記名投票" + 0.002*"議長" + 0.001*"買票" + 0.001*"賄選" + 0.001*"記名" + 0.001*"黨意" + 0.001*"第六十六" + 0.001*"增修" + 0.001*"小黨" + 0.000*"別有" + 0.000*"修為" + 0.000*"二十九條" + 0.000*"寶座" + 0.000*"行之" + 0.000*"本文" + 0.000*"強人" + 0.000*"更常" + 0.000*"少數黨" + 0.000*"傳聞"

strTM =  30.003*"純敬" + 0.000*"代表大會" + 0.000*"活躍" + 0.000*"實是" + 0.000*"定於" + 0.000*"定到" + 0.000*"退黨" + 0.000*"二十幾個" + 0.000*"久遠" + 0.000*"票率" + 0.000*"大黨" + 0.000*"領袖" + 0.000*"

In [36]:
import pyLDAvis
import pyLDAvis.gensim #import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

In [37]:
pyLDAvis.enable_notebook
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict, doc_topic_dist=None)
"""
参数说明：
prepare(topic_model, corpus, dictionary, doc_topic_dist=None)
topic_model：训练得到的 gensim LdaModel 对象，不支持其他 gensim 模型类型
corpus：以词袋形式表示的语料（用于训练主题模型的语料）
dictionary：用于构建词袋模型的字典，即 gensim Dictionary 对象
doc_topic_dist：可选参数，用于传入 LDA 模型的文档主题分布，默认取值为 None，当需要多次调用 prepare 函数时，可以传入该参数
"""
pyLDAvis.display(vis)

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
